In [1]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os


In [2]:
# BrainFlow Parameters
params = BrainFlowInputParams()
params.serial_port = 'COM3'  # Update based on your device and OS
board_id = 38  # Replace with your device ID

# Prepare the board session
try:
    board = BoardShim(board_id, params)
    board.prepare_session()
    print("Successfully prepared the physical board.")
except Exception as e:
    print("Error initializing the board:", e)
    print("Using synthetic board instead.")
    board_id = BoardIds.SYNTHETIC_BOARD
    board = BoardShim(board_id, params)
    board.prepare_session()


[2024-11-16 11:19:41.155] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "COM3",
    "timeout": 0
}
[2024-11-16 11:19:41.156] [board_logger] [info] Use timeout for discovery: 6
[2024-11-16 11:19:46.469] [board_logger] [info] found 1 BLE adapter(s)
[2024-11-16 11:19:47.459] [board_logger] [warning] Probably bluetooth is disabled.


[WARNING] SimpleBLE: /Users/runner/work/brainflow/brainflow/third_party/SimpleBLE/simpleble/src/frontends/base/Adapter.cpp:46 in scan_start: Bluetooth is not enabled.


In [ ]:
# Start Streaming
print("Starting stream...")
board.start_stream()
time.sleep(10)  # Stream for 10 seconds
data = board.get_board_data()  # Retrieve all data from the board
print("Stream stopped.")
board.stop_stream()
board.release_session()

# Isolate EEG Channels
eeg_channels = board.get_eeg_channels(board_id)
eeg_data = data[eeg_channels]
print(f"EEG Data Shape: {eeg_data.shape}")

# Save EEG Data to CSV
output_file = "eeg_data_test.csv"
DataFilter.write_file(eeg_data, output_file, 'w')
print(f"EEG data saved to '{output_file}'")


In [ ]:
# Function to analyze steady periods and fluctuations
def analyze_brainwave_activity(data, steady_threshold=5.0, fluctuation_threshold=50.0):
    results = []
    for channel_idx in range(data.shape[0]):
        channel_data = data[channel_idx]
        steady_segments = []
        fluctuation_comments = []

        # Analyze steady periods
        start_idx = 0
        for i in range(1, len(channel_data)):
            diff = abs(channel_data[i] - channel_data[i - 1])
            if diff > steady_threshold:
                if i - 1 > start_idx:
                    steady_segments.append((start_idx, i - 1))
                start_idx = i

        # Handle the last segment
        if start_idx < len(channel_data) - 1:
            steady_segments.append((start_idx, len(channel_data) - 1))

        # Analyze large fluctuations
        for i in range(1, len(channel_data)):
            diff = abs(channel_data[i] - channel_data[i - 1])
            if diff > fluctuation_threshold:
                fluctuation_comments.append(
                    f"Channel {channel_idx + 1}: Huge fluctuation from {channel_data[i - 1]:.2f} µV to {channel_data[i]:.2f} µV at sample {i}."
                )

        # Store results
        results.append({
            "channel": channel_idx + 1,
            "steady_segments": steady_segments,
            "fluctuations": fluctuation_comments,
        })

    return results


In [ ]:
# Load EEG Data from CSV
if not os.path.exists(output_file):
    raise FileNotFoundError(f"The file '{output_file}' was not found.")
eeg_data = DataFilter.read_file(output_file)
eeg_data = np.array(eeg_data)

# Analyze EEG Data
analysis_results = analyze_brainwave_activity(eeg_data)

# Save Analysis Results to CSV
steady_segments_df = pd.DataFrame([
    {"channel": r["channel"], "start": s[0], "end": s[1]}
    for r in analysis_results for s in r["steady_segments"]
])
steady_segments_df.to_csv("steady_segments.csv", index=False)
print("Steady segments saved to 'steady_segments.csv'")

fluctuation_comments_df = pd.DataFrame([
    {"channel": r["channel"], "comment": c}
    for r in analysis_results for c in r["fluctuations"]
])
fluctuation_comments_df.to_csv("fluctuation_comments.csv", index=False)
print("Fluctuation comments saved to 'fluctuation_comments.csv'")


In [ ]:
# Visualize EEG Data
offsets = [i * 100 for i in range(eeg_data.shape[0])]  # Offset each channel for visualization
plt.figure(figsize=(12, 6))
for i in range(eeg_data.shape[0]):
    plt.plot(np.arange(eeg_data.shape[1]), eeg_data[i] + offsets[i], label=f"Channel {i+1}")

plt.xlabel('Time')
plt.ylabel('Amplitude (with offsets)')
plt.legend()
plt.title('EEG Data Visualization')
plt.show()
